In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ In

In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jzbu7kfy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jzbu7kfy
  Resolved https://github.com/huggingface/transformers to commit 816f4424964c1a1631e303b663fc3d68f731e923
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9991549 sha256=9fa98f60ed04988769d18424db257729da798b3ca451dcf692897cfdeb3e2d45
  Stored in directory: /tmp/pip-ephem-wheel-cache-1tg69k1z/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name_or_path = "openai/whisper-large-v2"
language = "Nepali"
language_abbr = "ne-NP"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_17_0"

In [3]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name,language_abbr, split="train")
common_voice["test"] = load_dataset(dataset_name,language_abbr,  split="test")

print(common_voice)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 283
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 217
    })
})


In [4]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [6]:
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)


In [7]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/79a3ac66de6f0867932eeaafdcd73767084326edfc3772c28bae0a2bba29c2b4/ne-NP_train_0/common_voice_ne-NP_35314089.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        7.00167766e-06, -4.02070254e-05, -3.65305859e-05]), 'sampling_rate': 48000}, 'sentence': 'म पनि जान्छु है त अहिले लाई ।', 'variant': ''}


In [8]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [9]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/79a3ac66de6f0867932eeaafdcd73767084326edfc3772c28bae0a2bba29c2b4/ne-NP_train_0/common_voice_ne-NP_35314089.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        9.42416955e-06,  5.37711458e-06, -5.44977956e-06]), 'sampling_rate': 16000}, 'sentence': 'म पनि जान्छु है त अहिले लाई ।', 'variant': ''}


In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [11]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"],num_proc=2)
#num_proc=2


In [12]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 283
})

In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [15]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     wer = 100 * metric.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}

In [ ]:
# from transformers import WhisperForConditionalGeneration, BitsAndBytesConfig

# model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True))

# model.hf_device_map - this should be {" ": 0}

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
# model.config.forced_decoder_ids = None
# model.config.suppress_tokens = []

In [18]:
!pip install peft==0.9.0


In [20]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model, )

In [ ]:
# from peft import prepare_model_for_kbit_training

# model = prepare_model_for_kbit_training(model)

In [21]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [22]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 15,728,640 || all params: 1,559,033,600 || trainable%: 1.0088711365810203


In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="girikabin/whisper-NEPALI-LARGE-v2-common-17",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    eval_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],
    push_to_hub=True
)

In [28]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False

<ipython-input-28-91e6801b3f13>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [29]:
processor.save_pretrained(training_args.output_dir)

[]

In [30]:
from accelerate import Accelerator

In [31]:
accelerator = Accelerator()
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

Step,Training Loss,Validation Loss
25,1.553000,0.838802
50,0.508000,0.445582
75,0.327600,0.398426
100,0.160000,0.390401


TrainOutput(global_step=108, training_loss=0.6047122059045015, metrics={'train_runtime': 1948.3049, 'train_samples_per_second': 0.436, 'train_steps_per_second': 0.055, 'total_flos': 1.8218014027776e+18, 'train_loss': 0.6047122059045015, 'epoch': 3.0})

In [32]:
peft_model_id = "girikabin/whisper-NEPALI-LARGE-v2-common-17"
model.push_to_hub(peft_model_id)

IsADirectoryError: [Errno 21] Is a directory: 'girikabin/whisper-NEPALI-LARGE-v2-common-17'

In [ ]:
# kwargs = {
#     "dataset_tags": "mozilla-foundation/common_voice_17_0",
#     "dataset": "Common Voice 17.0 and Common Voice 15.0",  # a 'pretty' name for the training dataset
#     "dataset_args": "config: hi, split: test",
#     "language": "hi",
#     "model_name": "Whisper Large v2 Hi - Kabin",  # a 'pretty' name for your model
#     "finetuned_from": "openai/whisper-large-v2",
#     "tasks": "automatic-speech-recognition",
# }

In [ ]:
# trainer.push_to_hub(**kwargs)

In [ ]:
from peft import PeftConfig

peft_model_id = "girikabin/whisper-NEPALI-LARGE-v3-using-common-15-and-17-v4"
peft_config = PeftConfig.from_pretrained(peft_model_id)

# Print the base model name or path
print("Base model name or path:", peft_config.base_model_name_or_path)


Base model name or path: openai/whisper-large-v3


In [33]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "/content/girikabin/whisper-NEPALI-LARGE-v2-common-17"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)

NameError: name 'BitsAndBytesConfig' is not defined

In [37]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "/content/girikabin/whisper-NEPALI-LARGE-v2-common-17" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


KeyboardInterrupt: 

In [51]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_17_0",
    "dataset": "Common Voice 17.0"  ,
    "dataset_args" : "config: hi, split: test",
    "language": "nep",
    "model_name": "Whisper Large v2 Hi - Kabin",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-large-v2",
    "tasks": "automatic-speech-recognition",
}

In [52]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v2-common-17/commit/7d1b1ed23639bdf45626a33308350663918a85ad', commit_message='End of training', commit_description='', oid='7d1b1ed23639bdf45626a33308350663918a85ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v2-common-17', endpoint='https://huggingface.co', repo_type='model', repo_id='girikabin/whisper-NEPALI-LARGE-v2-common-17'), pr_revision=None, pr_num=None)

In [35]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

  0%|          | 0/28 [00:00<?, ?it/s]<ipython-input-35-1a5240d5c004>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 28/28 [10:40<00:00, 22.87s/it]

wer=56.72804532577904 and normalized_wer=26.113951789627464
{'eval/wer': 56.72804532577904, 'eval/normalized_wer': 26.113951789627464}


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")

  0%|          | 0/18 [00:00<?, ?it/s]<ipython-input-95-25fc43e49d9b>:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 18/18 [09:38<00:00, 32.13s/it]

wer=65.3475935828877


In [ ]:
print(f"{wer=}")

wer=65.3475935828877


##Using AutomaticSpeechRecognitionPipeline

In [ ]:
import torch
import gradio as gr
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig


peft_model_id = "girikabin/whisper-NEPALI-LARGE-v3-using-common-15-and-17-v4"
language = "Nepali"
task = "transcribe"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
pipe = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text


iface = gr.Interface(
    fn=transcribe,  # Assuming 'transcribe' is your function that processes the audio input
    inputs=gr.Audio(type="filepath"),  # No need for source="microphone", it's implied
    outputs="text",
    title="PEFT LoRA + INT8 Whisper Large V2 Nepali By Kabin Giri",
    description="Realtime demo for Nepali speech recognition using `PEFT-LoRA+INT8` fine-tuned Whisper Large V2 model.",
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef5eb1b95fb1410cdf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [41]:
import shutil

# Replace 'whisper-NEPALI-LARGE-v2' with your actual folder path
shutil.make_archive("/content/girikabin/whisper-NEPALI-LARGE-v2-common-17", 'zip', "whisper-NEPALI-LARGE-v2-26")
# from google.colab import files

# # Replace 'whisper-NEPALI-LARGE-v2.zip' with your actual zip file name
# files.download("whisper-NEPALI-LARGE-v2.zip")



'/content/girikabin/whisper-NEPALI-LARGE-v2-common-17.zip'

In [40]:
from google.colab import files

# Replace 'whisper-NEPALI-LARGE-v2.zip' with your actual zip file name
files.download("/content/girikabin/whisper-NEPALI-LARGE-v2-common-17.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:

from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/content/girikabin/whisper-NEPALI-LARGE-v2-common-17",
    repo_id="girikabin/whisper-NEPALI-LARGE-v2-common-17",
    repo_type="model",
)

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

optimizer.pt:   0%|          | 0.00/126M [00:00<?, ?B/s]

events.out.tfevents.1729403477.01984e13a546.8050.0:   0%|          | 0.00/9.58k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v2-common-17/commit/25e34908d16d317f01a38df745e74f9bb59370af', commit_message='Upload folder using huggingface_hub', commit_description='', oid='25e34908d16d317f01a38df745e74f9bb59370af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v2-common-17', endpoint='https://huggingface.co', repo_type='model', repo_id='girikabin/whisper-NEPALI-LARGE-v2-common-17'), pr_revision=None, pr_num=None)